# About 

This notebook demonstrates classifiers, which are provided by __Reproducible experiment platform (REP)__ package. <br /> REP contains following classifiers
* __scikit-learn__
* __TMVA__ 
* __XGBoost__ 

### In this notebook we show the most simple way to
* train classifier
* build predictions 
* measure quality

# Loading data

In [1]:
import numpy, pandas
from rep.utils import train_test_split
from sklearn.metrics import roc_auc_score

sig_data = pandas.read_csv('toy_datasets/toyMC_sig_mass.csv', sep='\t')
bck_data = pandas.read_csv('toy_datasets/toyMC_bck_mass.csv', sep='\t')

labels = numpy.array([1] * len(sig_data) + [0] * len(bck_data))
data = pandas.concat([sig_data, bck_data])

### First rows of our data

In [2]:
data[:5]

,CDF1,CDF2,CDF3,DOCAone,DOCAthree,DOCAtwo,FlightDistance,FlightDistanceError,Hlt1Dec,Hlt2Dec,...,p1_IP,p1_IPSig,p1_Laura_IsoBDT,p1_pt,p2_IP,p2_IPSig,p2_Laura_IsoBDT,p2_pt,peakingbkg,pt
0,1.000000,1.000000,1.000000,0.111337,0.012695,0.123426,162.650955,0.870942,0,0,...,11.314665,83.196968,-0.223668,699.066467,9.799975,64.790207,-0.121159,521.628174,NaN,220.742111
1,0.759755,0.597375,0.389256,0.021781,0.094551,0.088421,4.193265,1.262280,0,0,...,0.720070,7.237868,-0.256142,587.628935,0.882111,8.834325,-0.203220,532.679950,NaN,661.208843
2,1.000000,0.796142,0.566286,0.011852,0.004400,0.009153,1.580610,0.261697,0,0,...,0.362181,4.173097,-0.252788,802.746495,0.427290,5.008959,-0.409469,674.122342,NaN,1290.963982
3,0.716397,0.524712,0.279033,0.015171,0.083900,0.069127,7.884569,1.310151,0,0,...,0.753449,6.615949,-0.253550,564.203857,0.917409,8.695459,-0.192284,537.791687,NaN,692.654175
4,1.000000,0.996479,0.888159,0.005547,0.070438,0.064689,-2.267649,0.139555,0,0,...,0.589455,21.869143,-0.254778,746.624928,0.388996,8.465344,-0.217319,988.539221,NaN,1328.837840


### Splitting into train and test

In [3]:
# Get train and test data
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, train_size=0.5)

# Classifiers

All classifiers inherit from __sklearn.BaseEstimator__ and have the following methods:
    
* `classifier.fit(X, y, sample_weight=None)` - train classifier
        
* `classifier.predict_proba(X)` - return probabilities vector for all classes

* `classifier.predict(X)` - return predicted labels

* `classifier.staged_predict_proba(X)` - return probabilities after each iteration (not supported by TMVA)

* `classifier.get_feature_importances()`


Here we use `X` to denote matrix with data of shape `[n_samples, n_features]`, `y` is vector with labels (0 or 1) of shape [n_samples], <br /> `sample_weight` is vector with weights.


## Difference from default scikit-learn interface
X should be* `pandas.DataFrame`, `not numpy.array`. <br />
Provided this, you'll be able to choose features used in training by setting e.g. `features=['FlightTime', 'p']` in constructor.

\* it works fine with `numpy.array` as well, but in this case all the features will be used.

# Variables used in training

In [4]:
variables = ["FlightDistance", "FlightDistanceError", "IP", "VertexChi2", "pt", "p0_pt", "p1_pt", "p2_pt", 'LifeTime','dira']

# Sklearn
wrapper for scikit-learn classifiers. In this example we use GradientBoosting with default settings

In [5]:
from rep.estimators import SklearnClassifier
from sklearn.ensemble import GradientBoostingClassifier
# Using gradient boosting with default settings
sk = SklearnClassifier(GradientBoostingClassifier(), features=variables)
# Training classifier
sk.fit(train_data, train_labels)
print('training complete')

training complete


### Predicting probabilities, measuring the quality

In [6]:
# predict probabilities for each class
prob = sk.predict_proba(test_data)
print prob

[[ 0.01925103  0.98074897]
 [ 0.01577953  0.98422047]
 [ 0.01379252  0.98620748]
 ..., 
 [ 0.01624765  0.98375235]
 [ 0.01442193  0.98557807]
 [ 0.03089716  0.96910284]]


In [7]:
print 'ROC AUC', roc_auc_score(test_labels, prob[:, 1])

ROC AUC 0.908278717567


### Predictions of classes

In [8]:
sk.predict(test_data)

array([1, 1, 1, ..., 1, 1, 1])

In [9]:
sk.get_feature_importances()

,effect
FlightDistance,0.018544
FlightDistanceError,0.075000
IP,0.163375
VertexChi2,0.101842
pt,0.098344
p0_pt,0.061443
p1_pt,0.109733
p2_pt,0.091812
LifeTime,0.127267
dira,0.152638


## TMVA

__TMVAClassifier__ wraps classifiers from TMVA (CERN library for machine learning)

    Parameters:
    -----------
    :param str method: algorithm method (default='kBDT')
    :param list(str) | None features: features used in training
    :param str factory_options: default="!V:!Silent:Color:Transformations=I;D;P;G,D:AnalysisType=Classification"
    :param kwargs: other parameters passed as key=value.

TMVA __doesn't__ support `staged_predict_proba`

In [10]:
from rep.estimators.tmva import TMVAClassifier, TMVARegressor

In [11]:
from rep.estimators import TMVAClassifier

In [12]:
tmva = TMVAClassifier(method='kBDT', NTrees=50, Shrinkage=0.05, features=variables)
tmva.fit(train_data, train_labels)
print('training complete')

training complete


### Predict probabilities and estimate quality

In [13]:
# predict probabilities for each class
prob = tmva.predict_proba(test_data)
print prob

[[ 0.2709105   0.7290895 ]
 [ 0.28991175  0.71008825]
 [ 0.17888081  0.82111919]
 ..., 
 [ 0.27305794  0.72694206]
 [ 0.22307611  0.77692389]
 [ 0.31567514  0.68432486]]


In [14]:
print 'AUC', roc_auc_score(test_labels, prob[:, 1])

AUC 0.901148306027


In [15]:
# predict labels
tmva.predict(test_data)

array([1, 1, 1, ..., 1, 1, 1])

## XGBoost

XGBoost is open-source library with fast gradient boosting over decision trees.
    Parameters:
    -----------
    :param list(str) features: list of features to train model

    :param n_estimators: the number of round for boosting.

    :param nthreads: number of parallel threads used to run xgboost.

    :param num_feature: feature dimension used in boosting, set to maximum dimension of the feature
    (set automatically by xgboost, no need to be set by user).

    :param gamma: minimum loss reduction required to make a further partition on a leaf node of the tree.
     The larger, the more conservative the algorithm will be.

    :param eta: step size shrinkage used in update to prevents overfitting.
    After each boosting step, we can directly get the weights of new features
     and eta actually shrinkage the feature weights to make the boosting process more conservative.

    :param max_depth: maximum depth of a tree.

    :params scale_pos_weight: ration of weights of the class 1 to the weights of the class 0.

    :param min_child_weight: minimum sum of instance weight(hessian) needed in a child.
    If the tree partition step results in a leaf node with the sum of instance weight less than min_child_weight,
    then the building process will give up further partitioning.
    In linear regression mode, this simply corresponds to minimum number of instances needed to be in each node.
    The larger, the more conservative the algorithm will be.

    :param subsample: subsample ratio of the training instance.
    Setting it to 0.5 means that XGBoost randomly collected half of the data instances to grow trees
        and this will prevent overfitting.

    :param colsample: subsample ratio of columns when constructing each tree.

    :param objective: specify the learning task and the corresponding learning objective, and the options are below:
    "reg:linear" --linear regression
    "reg:logistic" --logistic regression
    "binary:logistic" --logistic regression for binary classification, output probability
    "binary:logitraw" --logistic regression for binary classification, output score before logistic transformation
    "multi:softmax" --set XGBoost to do multiclass classification using the softmax objective, you also need to
        set num_class(number of classes)
    "multi:softprob" --same as softmax, but output a vector of ndata * nclass, which can be further reshaped to ndata,
        nclass matrix. The result contains predicted probability of each data point belonging to each class.
    "rank:pairwise" --set XGBoost to do ranking task by minimizing the pairwise loss

    :param base_score: the initial prediction score of all instances, global bias.

    :param random_state: random number seed.

    :param verbose: if 1, will print messages during training

    :param missing: the number considered by xgboost as missing value.

In [16]:
from rep.estimators import XGBoostClassifier
# XGBoost with default parameters
xgb = XGBoostClassifier(features=variables)
xgb.fit(train_data, train_labels, sample_weight=numpy.ones(len(train_labels)))
print('training complete')

training complete


/Users/antares/code/xgboost/wrapper/xgboost.py:80: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if label != None:
/Users/antares/code/xgboost/wrapper/xgboost.py:82: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if weight !=None:


### Predict probabilities and estimate quality

In [17]:
prob = xgb.predict_proba(test_data)
print 'ROC AUC:', roc_auc_score(test_labels, prob[:, 1])

ROC AUC: 0.922402718668


### Predict labels

In [18]:
xgb.predict(test_data)

array([1, 1, 1, ..., 1, 1, 1])

In [19]:
xgb.get_feature_importances()

,effect
FlightDistance,1012
FlightDistanceError,940
IP,934
VertexChi2,918
pt,998
p0_pt,866
p1_pt,1070
p2_pt,1040
LifeTime,414
dira,338


# Advantages of common interface

As one can see above, all the classifiers implement the same interface, 
this simplifies work, simplifies comparison of different classifiers, 
but this is not the only profit. 

`Sklearn` provides different tools to combine different classifiers and transformers. 
One of this tools is `AdaBoost`, which is abstract metaclassifier built on the top of some other classifier (usually, decision dree)

Let's show that now you can run AdaBoost over classifiers from other libraries! <br />
_(isn't boosting over neural network what you were dreaming of all your life?)_

## AdaBoost over TMVA classifier

In [20]:
from sklearn.ensemble import AdaBoostClassifier

# Construct AdaBoost with TMVA as base estimator
base_tmva = TMVAClassifier(method='kBDT', NTrees=15, Shrinkage=0.05)
ada_tmva  = SklearnClassifier(AdaBoostClassifier(base_estimator=base_tmva, n_estimators=5), features=variables)
ada_tmva.fit(train_data, train_labels)
print('training complete')

training complete


In [21]:
prob = ada_tmva.predict_proba(test_data)
print 'AUC', roc_auc_score(test_labels, prob[:, 1])

AUC 0.885266565878


## AdaBoost over XGBoost

In [22]:
# Construct AdaBoost with xgboost base estimator
base_xgb = XGBoostClassifier(n_estimators=50)
# ada_xgb = SklearnClassifier(AdaBoostClassifier(base_estimator=base_xgb, n_estimators=1), features=variables)
ada_xgb = AdaBoostClassifier(base_estimator=base_xgb, n_estimators=1)
ada_xgb.fit(train_data[variables], train_labels)
print('training complete!')

training complete!


In [23]:
# predict probabilities for each class
prob = ada_xgb.predict_proba(test_data[variables])
print 'AUC', roc_auc_score(test_labels, prob[:, 1])

AUC 0.917898066693


In [24]:
# predict probabilities for each class
prob = ada_xgb.predict_proba(train_data[variables])
print 'AUC', roc_auc_score(train_labels, prob[:, 1])

AUC 0.950702225833


# Other advantages of common interface
There are many things you can do with classifiers now: 
* cloning
* getting / setting parameters as dictionaries 
* do automatic hyperparameter optimization 
* build pipelines (`sklearn.pipeline`)
* use hierarchical training, training on subsets
* passing over internet / train classifiers on other machines

And you can replace classifiers at any moment.

# Exercises

### Exercise 1. Play with parameters in each type of classifiers

### Exercise 2. Add weight column and train models with weights in training